<a href="https://colab.research.google.com/github/saraoud-github/Fingerprint-Authentication/blob/master/Fingerprint_Authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install np_utils
!pip install hyperas



In [ ]:
keras.__version__

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
#Checks number of images in drive
train_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory('gdrive/MyDrive/Datasets/Dataset/Classes')
                                      

Found 11161 images belonging to 5 classes.


In [ ]:
# Load data
data_dir = 'gdrive/MyDrive/Datasets/Dataset/Classes'

categories = ['A', 'L', 'R', 'T', 'W']
data = []
labels = []
img_height = 256
img_width = 256


for category in categories:
  path = os.path.join(data_dir, category)
  label = categories.index(category)

  for img in os.listdir(path):
    labels.append(label)
    imgpath = os.path.join(path, img)
    img = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_height, img_width))
    data.append(img)

data_X, X_test, data_Y, Y_test= train_test_split(data, labels, test_size=0.05, random_state=12345)
X_train, X_val, Y_train, Y_val = train_test_split(data_X, data_Y, test_size=0.2, random_state=12345)

X_train_len = len(X_train)
X_val_len = len(X_val)

X_train = np.asarray(X_train)
X_val = np.array(X_val)

Y_train = np.asarray(Y_train)
Y_val = np.asarray(Y_val)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_val /= 255
nb_classes = 5
Y_train = keras.utils.np_utils.to_categorical(Y_train, nb_classes)
Y_val = keras.utils.np_utils.to_categorical(Y_val, nb_classes)


In [ ]:
img_height = 256
img_width = 256

cnn2 = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(img_height,img_width,1)),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

print(cnn2.summary())


with tf.device('/device:GPU:0'):
  cnn2.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
  cnn2.fit(X_train, Y_train,batch_size=32, epochs=23, verbose=2, validation_data=(X_val, Y_val))



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 128)     

In [ ]:
cnn2.save('ml-model')

INFO:tensorflow:Assets written to: ml-model/assets


In [ ]:
!zip -r /content/ml-model.zip /content/ml-model

  adding: content/ml-model/ (stored 0%)
  adding: content/ml-model/saved_model.pb (deflated 89%)
  adding: content/ml-model/variables/ (stored 0%)
  adding: content/ml-model/variables/variables.index (deflated 66%)
  adding: content/ml-model/variables/variables.data-00000-of-00001 (deflated 38%)
  adding: content/ml-model/assets/ (stored 0%)
  adding: content/ml-model/keras_metadata.pb (deflated 92%)


In [ ]:
from google.colab import files
files.download('/content/ml-model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(len(X_test))
print(len(Y_test))


X_test = np.asarray(X_test)
X_test = X_test.astype('float32')
X_test /= 255

nb_classes = 5
Y_test = keras.utils.np_utils.to_categorical(Y_test, nb_classes)

score, acc = cnn2.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', acc)
print('Test score:', score)

559
559
Test accuracy: 0.8336314558982849
Test score: 0.9865067601203918
